# Prescriptive part of the problem: metaheuristic stochastic approach

Optimization problem to solve: 
maximize total profit on the portfolio by finding personalized rebate/surcharge coefficients while maintaining a certain percentage of customers in the portfolio.

**Note: for the rest of this study, we will search personalized surcharge coefficients** ```delta``` **between -5% and 10% for each customer**

Difficulties:
- large scale problem (one variable to optimize per client i: the rebate/surcharge coefficient delta_i)
- nonlinear problem, both in the objective (profit_i = retention_proba_i(delta_i) * [premium_i * (1+delta_i) - cost_i]) and the global constraint (avg(retention_proba_i for i in range(N)) > percentage_to_keep)

Metaheuristic stochastic resolution:
- run a genetic algorithm for many generations to find good quality solutions and compare them with the divide & conquer deterministic approach.

Package importations

In [ ]:
import pandas as pd
import sys
import numpy
import os
import matplotlib.pyplot as plt
from copy import deepcopy, copy
import random
from deap import creator, base, tools, algorithms

Data importation

In [ ]:
os.chdir(r"### wd ###")
contract_df = pd.read_csv("data.csv")
contract_df = contract_df.set_index("id_police").dropna()

We define right below the general hyperparameters of the genetic algorithm.

In [ ]:
# general parameters
min_retention_rate = 0.93
delta_min = -0.05
delta_max = 0.1
mating_change_rate=0.3
penality_distance=-1000000000
N=len(contract_df)
df = copy(contract_df)
prime_profit = df['prime_profit']
pcc = df['pcc']
A = df['coeff_non_prix']
B = df['coeff_prix']

We build right below the individuals aka genes that will compose our population of solutions:

In [ ]:
def builder():
    '''Returns a vector of length N, initialized with a random delta'''
    return random.uniform(delta_min,delta_max)*numpy.ones(N)

# We first create an individual class, with the fitness attribute
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("delta0", builder)

# We initialize an individual with builder().
toolbox.register("individual", tools.initIterate, creator.individual, toolbox.delta0)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

We define right below the functions that will enable us to evaluate the fitness/quality of the individuals as well as their feasibility (ie check if the considered solution verifies the global constraint):

In [ ]:
def evalOneMax(delta):
    '''Evaluation function for an individual. 
    Returns the turnout associated with a delta individual.'''
    retention_proba = 1-(1/(1+numpy.exp(-A-100*pd.Series(delta)*B)))
    return (((prime_profit*(1+pd.Series(delta))-pcc)*retention_proba).sum(),)

def feasible(delta):
    """Feasibility function for the individual. Returns True if feasible False
    otherwise."""
    retention_proba = 1-(1/(1+numpy.exp(-A-100*pd.Series(delta)*B)))
    retention_rate=retention_proba.sum()/N
    if retention_rate>=min_retention_rate:
        return True
    return False

def distance(delta):
    """A distance function to the feasibility region."""
    retention_proba = 1-(1/(1+numpy.exp(-A-100*pd.Series(delta)*B)))
    retention_rate=retention_proba.sum()/N
    return (max(0,min_retention_rate-retention_rate))

We choose right below our strategies for mating, mutations, and selection:

In [ ]:
def maxmating(ind1,ind2):
    '''A mating function returning two individuals with some switched parameters. '''
    for i in range(len(ind1)):
        if random.random()<=mating_change_rate:
            ind1[i],ind2[i]=max(ind1[i],ind2[i]),min(ind1[i],ind2[i])
    return ind1,ind2

toolbox.register("evaluate", evalOneMax)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, penality_distance, distance))
# toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mate", maxmating)
# toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.2, low=delta_min, up=delta_max, indpb=0.60)
toolbox.register("mutate", tools.mutGaussian, mu=0,sigma=0.05, indpb=0.50)
toolbox.register("select", tools.selTournament, tournsize=3)

The main algorithm that loops over the population:

In [ ]:
def main():
    # Genetic algorithm
    pop = toolbox.population(n=100)
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    g = 0
    # Begin the evolution
    while g < 20:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        # Apply crossover and mutation on the offspring
        for (child1, child2) in list(zip(offspring[::2], offspring[1::2])):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
         # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        pop[:] = offspring
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        maxi=max(fits)
        print("  Min %s" % min(fits))
        print("  Max %s" % maxi)
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    return maxi

Do some statistics over the stochastic results:

In [ ]:
def main2(k):
    # Create a list in order to do statistics
    l=[]
    for i in range(k):
        print("Tour{}.".format(i+1))
        l+=[main()]

    return l
sample=plt.boxplot(main2(k))